# Logs recovering

## Content

* [Task](#Task)
* [Dataset](#Dataset)
* [Model architecture](#Architecture)
* [Training](#Training)
* [Inference](#Inference)

## Task

Recover missed logs on the basis of known measurements

## Dataset


Logs from one place were used to train and estimate the model.

We choose the set of 12 logs measured for 90% of wells (321 wells) in the place: CFTC, CILD, GR, GZ1, GZ2, GZ3, GZ4, GZ7, LLD, NKTD, PROX и SP. This dataset can be represented as an array of 1d multichannel arrays of shape `[12, length]`, each array has a different length. The dataset was split into train and test in proportion 80:20. For each batch, we sample `n` uniformly from `range(3, 6)` and then split logs into `n` logs to recover and the remainder part as known logs. In the input crop, we pad channels that correspond to unknown logs by zeros and also create a binary mask with ones for unknown crops.

In [1]:
import os
import sys
sys.path.insert(0, os.path.join("..", "..", ".."))

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from well_logs import WellLogsDataset
from well_logs.batchflow import Pipeline, B, V
from well_logs.batchflow.models.torch import UNet
from well_logs.batchflow.models.tf.losses import softmax_cross_entropy, dice

In [2]:
WELLS = ['01b90c425a4175f1561707a2.npz',
 '044fdedce8205690c89575a1.npz',
 '2070fc1916b75860379a636b.npz',
 '207f8ed2b110a43f9ff1f03e.npz',
 '23886e95ab8c5eb5be302342.npz',
 '24574be1830c31b1856b0268.npz',
 '26f0edc7a368b9a63f0d4e7a.npz',
 '2c53ed6764a4410e70c8fdc3.npz',
 '34b30e76506ca5b836522fda.npz',
 '353fadfea3f536b05c3e899a.npz',
 '35a4a4828fe360c9764651cc.npz',
 '36d016abfca2077ad58325c5.npz',
 '3c238f32d263a24092e09963.npz',
 '3ea3c07a77d54f3c3a60370b.npz',
 '4105403378b4aba06fc8b3c8.npz',
 '427a680c3021797e32ef1f8c.npz',
 '42d9d1beca9f399355e9a26f.npz',
 '48e14b54912162db688e7a9b.npz',
 '54d9d0a88c0baa8857afeb60.npz',
 '57efacaa9fc7b3ca89fd0744.npz',
 '66aa890705908dda3ecd9640.npz',
 '687f0cc7350e002ae0cd76ad.npz',
 '6b665fc27b9f653e292fe1a8.npz',
 '733e534be4624b5bfe7aeb41.npz',
 '80d8e5d4740aad50cec4e831.npz',
 '8126cc6a41a16d0e996561ae.npz',
 '84988fabdde3a19f041768c3.npz',
 '8a3950e6cc2c8dbccc1950ce.npz',
 '9596b26c806c916b299c0905.npz',
 '9ac400488c3840e83f1aa2a1.npz',
 '9baa3f8ad097788e3a35bbc6.npz',
 '9eb6db54dec42f96473b1e66.npz',
 'a40668cc6dba8f49262270a0.npz',
 'a78c035caa6c6ecd5e15b603.npz',
 'a836b67a1bfcda06a0243d3c.npz',
 'aeb7fcfd92f9fce297fbd661.npz',
 'b260b78247e060fb21d3a1fa.npz',
 'bb16d8cc1383468d77d4c85c.npz',
 'c6621ae554989ffe13fc5260.npz',
 'cb6951e92410596f210112e0.npz',
 'ccce2ac4fd7fcd8d4893cf1c.npz',
 'cebd93c228d1c0e12b93b472.npz',
 'd6272c7c7acb970114475acc.npz',
 'd64515806c239707aa6434e2.npz',
 'd664829df76f48d589eb7c02.npz',
 'd9b1e421a8c95244b619317e.npz',
 'db68555d57523ed187fd1d47.npz',
 'dc640416bf3de60fa23cdeba.npz',
 'de8390668855c8419c8151c6.npz',
 'e509f739538b462b075dc749.npz',
 'e5cd694a58ac0a25435bd129.npz',
 'ecc3de5d827cd315db288d9a.npz',
 'ed68fee79945d2b206174881.npz',
 'ee4811155b47c41b9362e308.npz',
 'f07f5a3333e37530923cdf33.npz',
 'f4e253e44b38bd5ae58658e9.npz',
 'f88eddba8c02fd951a60978c.npz',
 'f8c625e848bfcf5210d1d1b0.npz',
 'f8ea33141e17f43db9d648b0.npz',
 'fbd217d6847d1010df40b0b8.npz',
 'fc090a98e6bfe3560aebd618.npz',
 'fca0490341d8a2672e75e6e3.npz',
 'fd056bdcb2bdfd8f77479fc3.npz']

LOGS = ('CILD', 'DT', 'GR', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ5', 'LLD', 'MINV', 'MNOR', 'MSFL',
 'NKTD', 'NKTR', 'PHIT', 'PRON', 'RT', 'SP')

In [3]:
WELL_NPZ_PATH = [os.path.join("/data/data/las/not_anon/", s) for s in WELLS]
ds = WellLogsDataset(path=WELL_NPZ_PATH, no_ext=True, sort=True)
ds.split(0.8, shuffle=42)

## Architecture

To train the model we use U-Net with custom input block.

Model was trained on crops of size 256 with zeros for unknown channels. Weighted binary mask with trainable weights is summed with crop and the result is an input of U-Net. Loss for the model is element-wise MSE computed for missed channels.

In [4]:
class InitialBlock(nn.Module):
    def __init__(self, input_shapes):
        super().__init__()
        n_channels = input_shapes[0][1]
        self.b = nn.Parameter(torch.tensor([1.] * n_channels, requires_grad=True))
        self.output_shape = input_shapes[0]

    def forward(self, inputs):
        x, masks = inputs
        z = masks * self.b
        return x + z[..., np.newaxis]

class MyLoss(nn.Module):
    def forward(self, predictions, target):
        target, mask = target
        mse_element = mask[..., np.newaxis] * ((predictions - target) ** 2)
        return torch.sum(mse_element) / mask.sum() / predictions.shape[-1]    

class CustomUNet(UNet):
    def initial_block(self, inputs, **kwargs):
        return InitialBlock(inputs)

Configuration of the model:
* input shape - [256, 12]
* output shape - [256, 12]
* filters in decoder and encoder - [64, 128, 256, 512, 1024].

In [15]:
CROP_SIZE = 256

model_config = {
    # 'device': 'cuda:3',
    'initial_block/inputs': ('x', 'masks'),
    'inputs/x': {'shape': [len(LOGS), CROP_SIZE]},
    'inputs/y': {'shape': [len(LOGS), CROP_SIZE]},
    'inputs/masks': {'shape': (len(LOGS), )},
    "head/num_classes": len(LOGS),
    'loss': MyLoss
}

## Training

In [16]:
N_EPOCH = 100
BATCH_SIZE = 32

In [17]:
def tile_masks(masks, logs):
    new_masks = []
    for log, mask in zip(logs, masks):
        mask = mask[None, ...]
        new_masks.append(np.tile(mask, (log.shape[0], 1)))
    return new_masks

def sampler(n_channels, min_size=3, max_size=6):
    size = np.random.randint(min_size, max_size+1)
    return np.random.choice(np.arange(n_channels), size=size, replace=False)

def concatenate_batch(batch, model, concat_mask=True):
    masks = tile_masks(batch.masks, batch.missed_logs)
    missed_logs = np.concatenate(batch.missed_logs).astype('float32')
    masks = np.concatenate(masks).astype('float32')
    
    feed_dict = {"inputs": (missed_logs, masks)}
    if concat_mask:
        feed_dict["targets"] = (np.concatenate(batch.logs).astype('float32'),
                                masks)
    return feed_dict

All the training process is described in pipeline

In [18]:
template_train_ppl = (Pipeline()
    .load(fmt="npz", components=["dept", "logs"])
    .init_variable('loss', init_on_each_run=list)
    .keep_channels(tuple(LOGS))
    .drop_nans()
    .add_components("missed_logs", B("array_of_nones"))
    .add_components("masks", B("array_of_nones"))
    .standardize(components="logs")
    .copy_components('logs', 'missed_logs')
    .fill_channels('missed_logs', np.array([1, ]), 'masks')
    .random_crop(CROP_SIZE, n_crops=4, components=["logs", "missed_logs"])
    .init_model('dynamic', CustomUNet, 'model', model_config)
    .train_model('model', make_data=concatenate_batch, fetches='loss',
                 save_to=V('loss'), mode='a')
    .run(batch_size=BATCH_SIZE, n_epochs=N_EPOCH, shuffle=True, drop_last=True,
         lazy=True, bar=True)
)

In [19]:
train_ppl = (ds.train >> template_train_ppl)
train_ppl.run()



  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: max_pool1d() argument 'padding' should contain one int (got 2)

In [ ]:
train_loss = [np.mean(l) for l in np.array_split(train_ppl.get_variable("loss"), N_EPOCH)]

fig = plt.figure(figsize=(15, 4))
plt.plot(train_loss)
plt.xlabel("Epochs")
plt.ylabel("Training loss")
plt.show()

Here we can see that learning process converged.

## Inference

To check the quality of the algorithm we plot some examples of recovered logs.

In [ ]:
template_test_ppl = (Pipeline()
    .load(fmt="npz", components=["dept", "logs"])
    .init_variable('output', init_on_each_run=list)
    .init_variable('input', init_on_each_run=list)
    .drop_nans()
    .add_components("missed_logs", B("array_of_nones"))
    .add_components("masks", B("array_of_nones"))
    .standardize(components="logs")
    .copy_components('logs', 'missed_logs')
    .fill_channels('missed_logs', np.array([1, ]), 'masks')
    .random_crop(CROP_SIZE, n_crops=4, components=["logs", "missed_logs"])
    .import_model('model', train_ppl)
    .predict_model('model', make_data=concatenate_batch, fetches='predictions',
                   save_to=V('output'), mode='w')
    .update_variable('input', [B('missed_logs'), B('logs'), B('masks'), B('meta')], mode='w')
    .run(batch_size=BATCH_SIZE, n_epochs=N_EPOCH, shuffle=True, drop_last=True, lazy=True)
)

In [ ]:
test_ppl = (ds.test >> template_test_ppl)
test_ppl.next_batch()

In [ ]:
logs, target, masks, meta =  test_ppl.get_variable('input')

masks = np.concatenate(tile_masks(masks, logs))
logs = np.concatenate(logs)
target = np.concatenate(target)
output = test_ppl.get_variable('output')

In [ ]:
FULL_SET = LOGS#np.array(['CFTC', 'CILD', 'GR', 'GZ1', 'GZ2', 'GZ3', 'GZ4', 'GZ7', 'LLD', 'NKTD', 'PROX', 'SP'])

n_crops = 5

plt.figure(figsize=(5 * n_crops, 50))

random_indices = np.random.choice(range(len(logs)), n_crops, replace=False)

for i in range(12):
    for j, crop in enumerate(random_indices):
        ymax = max(logs[crop, i].max(), target[crop, i].max(), output[crop, i].max())
        ymin = min(logs[crop, i].min(), target[crop, i].min(), output[crop, i].min())

        plt.subplot(12, n_crops, n_crops * i + j + 1)
        if j == 0:
            plt.ylabel(FULL_SET[i])
        if i == 0:
            plt.title('Crop {}'.format(crop))
        if masks[crop, i]:
            plt.plot(target[crop, i], color='r', label='Unknown log')
        else:
            plt.plot(logs[crop, i], color='g', label='Known log')

        plt.plot(output[crop, i], color='b', label='Prediction')
        plt.ylim((ymin, ymax))
        plt.legend()